# Simulación del Truco

Las **cartas** deben registra el numero, palo y valor de la carta. 
El valor debe reflejar la jerarquía de las cartas en el juego del truco.

In [4]:
def calcular_valor(numero, palo):
    # Defino la variable jerarquía de las cartas para hacer el Truco
    jerarquia = {
        (1, 'espada'): 14,  # Ancho de espadas
        (1, 'basto'): 13,   # Ancho de bastos
        (7, 'espada'): 12,  # Siete de espadas
        (7, 'oro'): 11,     # Siete de oros
        (3, 'espada'): 10, (3, 'basto'): 10, (3, 'oro'): 10, (3, 'copa'): 10,  # Tres
        (2, 'espada'): 9, (2, 'basto'): 9, (2, 'oro'): 9, (2, 'copa'): 9,      # Dos
        (1, 'oro'): 8, (1, 'copa'): 8,                                          # Falsos
        (12, 'espada'): 7, (12, 'basto'): 7, (12, 'oro'): 7, (12, 'copa'): 7,  # Doce
        (11, 'espada'): 6, (11, 'basto'): 6, (11, 'oro'): 6, (11, 'copa'): 6,  # Once
        (10, 'espada'): 5, (10, 'basto'): 5, (10, 'oro'): 5, (10, 'copa'): 5,  # Diez
        (7, 'basto'): 4, (7, 'copa'): 4,                                        # Siete falsos
        (6, 'espada'): 3, (6, 'basto'): 3, (6, 'oro'): 3, (6, 'copa'): 3,      # Seis
        (5, 'espada'): 2, (5, 'basto'): 2, (5, 'oro'): 2, (5, 'copa'): 2,      # Cinco
        (4, 'espada'): 1, (4, 'basto'): 1, (4, 'oro'): 1, (4, 'copa'): 1       # Cuatro
    }
    # Devuelve el valor de la carta según la jerarquía
    return jerarquia.get((numero, palo), 0)  # Si no está en el diccionario, devuelve 0

class Carta:
    def __init__(self, numero, palo):
        self.numero = numero
        self.palo = palo
        self.valor = calcular_valor(numero, palo)

    def __repr__(self):
        return f"Carta(numero={self.numero}, palo='{self.palo}', valor={self.valor})"

c = Carta(1, "oro")
c


Carta(numero=1, palo='oro', valor=8)

**Mazo**: Representan las cartas españolas de 1 al 12, en los palos de oro, copa, espada y basto.

Debe existir una función que permita barajar las cartas.
y otra función que permita repartir las cartas a los jugadores.

In [5]:
import random

class Mazo:
    def __init__(self):
        # Genero todas las combinaciones de números y palos
        numeros = list(range(1, 13))  # Números del 1 al 12
        palos = ['oro', 'copa', 'espada', 'basto']
        # Creo el mazo con todas las cartas posibles
        self.cartas = [Carta(numero, palo) for numero in numeros for palo in palos]
    
    def mezclar(self):
        # Mezclo las cartas del mazo
        random.shuffle(self.cartas)
    
    def repartir(self, cantidad):
        # Reparto la cantidad de cartas solicitadas
        if cantidad > len(self.cartas):
            return "No hay suficientes cartas en el mazo."
        # Tomo las primeras cantidad de cartas
        cartas_repartidas = self.cartas[:cantidad]
        # Elimino esas cartas del mazo
        self.cartas = self.cartas[cantidad:]
        return cartas_repartidas

m = Mazo()
m.mezclar()
print(m.repartir(3))


[Carta(numero=6, palo='copa', valor=3), Carta(numero=6, palo='espada', valor=3), Carta(numero=7, palo='espada', valor=12)]


**Jugador**: Deben tener un nombre y las cartas que tiene en la mano. Debe acumular los puntos ganados.

Nota: Para simplificar el problema, el jugador elige la carta a jugar de forma aleatoria.

In [10]:
class Jugador:
    def __init__(self, nombre):
        # Inicializa el jugador con un nombre, sin cartas y con 0 puntos
        self.nombre = nombre
        self.mano = []
        self.puntos = 0

    def recibir_cartas(self, cartas):
        # Agrega las cartas recibidas a la mano del jugador
        self.mano.extend(cartas)
    
    def jugar_carta(self):
        # Si el jugador no tiene cartas en la mano, no puede jugar
        if not self.mano:
            return "No hay cartas en la mano."
        # Elige una carta aleatoria de la mano
        carta_jugada = random.choice(self.mano)
        # Elimina la carta de la mano
        self.mano.remove(carta_jugada)
        return carta_jugada

j = Jugador("Juan")
j.recibir_cartas(m.repartir(3))
print(j.jugar_carta())

Carta(numero=12, palo='oro', valor=7)


**Partida**: Debe tener dos jugadores, un mazo de cartas y un puntaje. Debe permitir jugar una carta por ronda y definir el ganador de la ronda. Debe permitir jugar hasta que se terminen las cartas del mazo.

In [18]:
class Partida:
    def __init__(self, jugador1, jugador2):
        # Inicio los jugadores y el mazo
        self.jugador1 = jugador1
        self.jugador2 = jugador2
        self.mazo = Mazo()
        self.mazo.mezclar()  # Mezclo el mazo
        self.puntaje = {jugador1.nombre: 0, jugador2.nombre: 0}  # Puntaje de cada jugador

    def jugar_ronda(self):
        # Ambos jugadores juegan una carta
        carta_j1 = self.jugador1.jugar_carta()
        carta_j2 = self.jugador2.jugar_carta()

        # Verifico que ambos jugadores hayan podido jugar una carta
        if isinstance(carta_j1, str) or isinstance(carta_j2, str):
            return None, "No hay suficientes cartas para jugar una ronda."

        # Determino el ganador de la ronda comparando el valor de las cartas
        if carta_j1.valor > carta_j2.valor:
            ganador = self.jugador1.nombre
            self.puntaje[ganador] += 1
        elif carta_j1.valor < carta_j2.valor:
            ganador = self.jugador2.nombre
            self.puntaje[ganador] += 1
        else:
            ganador = None  # Empate

        return ganador, (carta_j1, carta_j2)

    def jugar(self):
        # Mientras que ningún jugador acumule 15 puntos
        while self.puntaje[self.jugador1.nombre] < 15 and self.puntaje[self.jugador2.nombre] < 15:
            # Reparte 3 cartas a cada jugador
            self.jugador1.recibir_cartas(self.mazo.repartir(3))
            self.jugador2.recibir_cartas(self.mazo.repartir(3))

            # Se juega hasta que ambos jugadores se queden sin cartas en la mano
            while self.jugador1.mano and self.jugador2.mano:
                ganador, cartas = self.jugar_ronda()
                if ganador:
                    print(f"{ganador} gana la ronda con {cartas}.")
                else:
                    print("Empate en la ronda con cartas:", cartas)

            # Si ya no quedan cartas en el mazo, terminamos la partida
            if len(self.mazo.cartas) == 0:
                print("No hay más cartas en el mazo. Fin de la partida.")
                break

        # Determina el ganador final de la partida
        if self.puntaje[self.jugador1.nombre] >= 15:
            print(f"¡{self.jugador1.nombre} gana la partida con {self.puntaje[self.jugador1.nombre]} puntos!")
        elif self.puntaje[self.jugador2.nombre] >= 15:
            print(f"¡{self.jugador2.nombre} gana la partida con {self.puntaje[self.jugador2.nombre]} puntos!")
        else:
            print("La partida termina en empate.")

p = Partida(Jugador("Juan"), Jugador("Pedro"))
p.jugar()

Juan gana la ronda con (Carta(numero=6, palo='espada', valor=3), Carta(numero=8, palo='espada', valor=0)).
Juan gana la ronda con (Carta(numero=11, palo='basto', valor=6), Carta(numero=10, palo='espada', valor=5)).
Pedro gana la ronda con (Carta(numero=4, palo='basto', valor=1), Carta(numero=5, palo='basto', valor=2)).
Empate en la ronda con cartas: (Carta(numero=8, palo='oro', valor=0), Carta(numero=9, palo='oro', valor=0))
Juan gana la ronda con (Carta(numero=12, palo='basto', valor=7), Carta(numero=11, palo='oro', valor=6)).
Juan gana la ronda con (Carta(numero=1, palo='espada', valor=14), Carta(numero=5, palo='copa', valor=2)).
Empate en la ronda con cartas: (Carta(numero=2, palo='oro', valor=9), Carta(numero=2, palo='copa', valor=9))
Pedro gana la ronda con (Carta(numero=11, palo='copa', valor=6), Carta(numero=3, palo='basto', valor=10)).
Juan gana la ronda con (Carta(numero=1, palo='copa', valor=8), Carta(numero=8, palo='basto', valor=0)).
Juan gana la ronda con (Carta(numero=6, 